![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/work-with-data/dataprep/how-to-guides/custom-python-transforms.png)

# Custom Python Transforms


There will be scenarios when the easiest thing for you to do is just to write some Python code. This SDK provides three extension points that you can use.

1. New Script Column
2. New Script Filter
3. Transform Partition

Each of these are supported in both the scale-up and the scale-out runtime. A key advantage of using these extension points is that you don't need to pull all of the data in order to create a dataframe. Your custom python code will be run just like other transforms, at scale, by partition, and typically in parallel.

## Initial data prep

We start by loading crime data.

In [ ]:
import azureml.dataprep as dprep
col = dprep.col

dflow = dprep.read_csv(path='../data/crime-spring.csv')
dflow.head(5)

We trim the dataset down and keep only the columns we are interested in. 

In [ ]:
dflow = dflow.keep_columns(['Case Number','Primary Type', 'Description', 'Latitude', 'Longitude'])
dflow = dflow.replace_na(columns=['Latitude', 'Longitude'], custom_na_list='')
dflow.head(5)

We look for null values using a filter. We found some, so now we'll look at a way to fill these missing values.

In [ ]:
dflow.filter(col('Latitude').is_null()).head(5)

## Transform Partition

We want to replace all null values with a 0, so we decide to use a handy pandas function. This code will be run by partition, not on all of the dataset at a time. This means that on a large dataset, this code may run in parallel as the runtime processes the data partition by partition.

In [ ]:
pt_dflow = dflow
dflow = pt_dflow.transform_partition("""
def transform(df, index):
    df['Latitude'].fillna('0',inplace=True)
    df['Longitude'].fillna('0',inplace=True)
    return df
""")
dflow.head(5)

### Transform Partition With File

Being able to use any python code to manipulate your data as a pandas DataFrame is extremely useful for complex and specific data operations that DataPrep doesn't handle natively. Though the code isn't very testable unfortunately, it's just sitting inside a string.
So to improve code testability and ease of script writing there is another transform_partiton interface that takes the path to a python script which must contain a function matching the 'transform' signature defined above.

The `script_path` argument should be a relative path to ensure Dataflow portability. Here `map_func.py` contains the same code as in the previous example.

In [ ]:
dflow = pt_dflow.transform_partition_with_file('../data/map_func.py')
dflow.head(5)

## New Script Column

We want to create a new column that has both the latitude and longitude. We can achieve it easily using [Data Prep expression](./add-column-using-expression.ipynb), which is faster in execution. Alternatively, We can do this using Python code by using the `new_script_column()` method on the dataflow. Note that we use custom Python code here for demo purpose only. In practise, you should always use Data Prep native functions as a preferred method, and use custom Python code when the functionality is not available in Data Prep. 

In [ ]:
dflow = dflow.new_script_column(new_column_name='coordinates', insert_after='Longitude', script="""
def newvalue(row):
    return '(' + row['Latitude'] + ', ' + row['Longitude'] + ')'
""")
dflow.head(5)

## New Script Filter

Now we want to filter the dataset down to only the crimes that incurred over $300 in loss. We can build a Python expression that returns True if we want to keep the row, and False to drop the row.

In [ ]:
dflow = dflow.new_script_filter("""
def includerow(row):
    val = row['Description']
    return 'OVER $ 300' in val
""")
dflow.head(5)